### 01. Loading Data

In [1]:
# Loading
import pandas as pd
data = pd.read_csv('../data/Dataset.csv')

In [2]:
# Columns Setting
data.columns = ['DATE','AED', 'AUD', 'BHD', 'CAD', 'CHF', 'CNH', 'DKK', 'EUR',
                'GBP', 'HKD', 'IDR(100)', 'JPY(100)', 'KWD', 'MYR', 'NOK', 'NZD', 'SAR',
                'SEK', 'SGD', 'THB', 'USD', 'BITCOIN', 'TETHER', 'KOSPI200', 'KOSDAQ',
                'SAMSUNG', 'S&P500', 'DOWJONES', 'NASDAQ', 'MICROSOFT', 'APPLE',
                'US_LIBOR', 'KOR_LIBOR', 'US_BOND_RATE_1YEARS', 'US_BOND_RATE_2YEARS',
                'US_BOND_RATE_10YEARS', 'HYUNDAI_CAR', 'IEF_US', 'GLD_US', 'HSI',
                'NAVER', 'SK_HYNIX', 'AMAZON', 'TESLA', 'NIKKEI', 'FTSE',
                'ETHEREUM', 'ETHEREUM_CLASSIC', 'EOS', 'RIPPLE', 'STELLA']

In [3]:
# Index Setting
data = data.set_index('DATE')

In [4]:
# Columns order
data = data[['USD', 'AED', 'AUD', 'BHD', 'CAD', 'CHF', 'CNH', 'DKK', 'EUR','GBP', 'HKD',
             'IDR(100)', 'JPY(100)', 'KWD', 'MYR', 'NOK', 'NZD', 'SAR', 'SEK', 'SGD', 'THB',
             
             'US_BOND_RATE_1YEARS', 'US_BOND_RATE_2YEARS', 'US_BOND_RATE_10YEARS',
             'US_LIBOR', 'KOR_LIBOR', 'IEF_US', 'GLD_US',
             
             'KOSPI200', 'KOSDAQ', 'S&P500', 'DOWJONES', 'NASDAQ', 'HSI', 'NIKKEI', 'FTSE',
            
             'MICROSOFT', 'APPLE', 'SAMSUNG', 'HYUNDAI_CAR', 'NAVER', 'SK_HYNIX', 'AMAZON', 'TESLA',
            
             'BITCOIN', 'TETHER', 'ETHEREUM', 'ETHEREUM_CLASSIC', 'EOS', 'RIPPLE', 'STELLA']]

### 03. Data Shift

In [5]:
# 데이터 shift 및 라벨링
data['USD_1'] = data['USD'].shift(-1)
data['Target'] = data['USD_1'] - data['USD'] * 1.001
data['Target'] = data['Target'].apply(lambda x : 'UP' if x > 0 else 'DOWN')
data = data.dropna()

In [6]:
# USD Omit
del data['USD']
del data['USD_1']

### 04. Train Test Split

In [7]:
train = data['2018-01-01':'2019-08-31']
valid = data['2019-09-01':'2019-10-31']
test = data['2019-11-01':'2019-12-31']

### 05. X, y

In [8]:
train_X = train[train.columns[:-1]]
train_y = train[train.columns[-1:]]

In [9]:
valid_X = valid[valid.columns[:-1]]
valid_y = valid[valid.columns[-1:]]

In [10]:
test_X = test[test.columns[:-1]]
test_y = test[test.columns[-1:]]

### 06. Reshape to Numpy Array : 4개의 데이터프레임을 모두 Numpy 배열로 변경.

In [11]:
import numpy as np
train_X = np.array(train_X)
train_y = np.array(train_y)
valid_X = np.array(valid_X)
valid_y = np.array(valid_y)
test_X = np.array(test_X)
test_y = np.array(test_y)

### 정규화

In [12]:
mean = train_X.mean(axis=0)
train_X -= mean
std = train_X.std(axis=0)
train_X /= std
valid_X -= mean
valid_X /= std
test_X -= mean
test_X /= std

In [13]:
# XGBoost Build
from xgboost import XGBClassifier
model = XGBClassifier(objective='binary:logistic', random_state=0)

In [14]:
# Model Fitting
model.fit(train_X, train_y, eval_set=[(valid_X, valid_y)], early_stopping_rounds=50,verbose=0)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [15]:
# Acc Check
from sklearn.metrics import accuracy_score
print('Train ACC = {:.4f}' .format(accuracy_score(train_y, model.predict(train_X))))
print('Test ACC = {:.4f}' .format(accuracy_score(test_y, model.predict(test_X))))

Train ACC = 0.6791
Test ACC = 0.4595


In [16]:
model.predict(test_X)

array(['UP', 'UP', 'UP', 'UP', 'UP', 'UP', 'UP', 'UP', 'UP', 'UP', 'UP',
       'UP', 'UP', 'UP', 'UP', 'UP', 'UP', 'UP', 'UP', 'UP', 'DOWN',
       'DOWN', 'DOWN', 'DOWN', 'DOWN', 'DOWN', 'DOWN', 'DOWN', 'DOWN',
       'DOWN', 'DOWN', 'UP', 'UP', 'UP', 'UP', 'UP', 'DOWN'], dtype=object)